In [2]:
from monai.utils import first, set_determinism, ensure_tuple
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    RandAffined,
    RandRotate90d,
    RandShiftIntensityd,
    RandFlipd,
    RandGaussianNoised,
    RandAdjustContrastd,
    ScaleIntensityRanged,
    Spacingd,
    EnsureTyped,
    EnsureType,
    Invertd,
    AddChanneld,
    RandGaussianSharpend,
    RandGaussianSmoothd,
    RandHistogramShiftd,
    OneOf,
    Rand3DElasticd,
    Rand3DElastic,
    RandGridDistortiond,
    RandSpatialCropSamplesd,
    FillHoles,
    LabelFilter,
    LabelToContour
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet, UNETR
from monai.networks.layers import Norm
from monai.metrics import DiceMetric, HausdorffDistanceMetric
from monai.losses import DiceLoss, DiceCELoss, DiceFocalLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch, ImageReader
from monai.data.image_reader import WSIReader
from monai.config import print_config, KeysCollection, PathLike
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
from numpy import random
from pathlib import Path
import re
from skimage import io
from typing import Optional, Union, Sequence, Callable, Dict, List
from monai.data.utils import is_supported_format
from monai. data.image_reader import _copy_compatible_dict, _stack_images
from nibabel.nifti1 import Nifti1Image
from PIL import Image
import numpy as np
from tqdm import tqdm
import pickle
import pandas as pd
from mlflow import log_metric, log_param, log_artifacts, set_experiment, start_run, end_run
import warnings
import argparse
warnings.filterwarnings('ignore')

In [16]:
#make list of data dictionaries
train_labels_path = Path('../../../rozakmat/TBI/PV-cap sep model/')#labels path
train_labels = list(train_labels_path.glob('*_sub1.tiff'))#get label images
print(train_labels)
train_labels = sorted([x.as_posix() for x in train_labels])#sort
train_images = [re.sub("_Simple Segmentation_sub1.tiff","_withOutput.tif",i) for i in train_labels]
train_images_paths = [Path(i) for i in train_images]
#combine images and labels into monai dictionary format
data_dicts = [
    {"image":image_name, "label":label_name}
    for image_name, label_name in zip(train_images,train_labels)
]

print(data_dicts)

mouse_ids_path = Path('/home/rozakmat/projects/rrg-bojana/rozakmat/TBI/raw')#each mouse has its own folder with raw data in it
mouse_ids = list(mouse_ids_path.glob('*'))#grab molder names/mouse ids
images = sorted([y.name for y in train_images_paths])#sort
#get mouse id corresponding to each image i have labels for
mouse_ids_with_raw_tiff = []
for i in mouse_ids:
    for j in images:
        j = re.sub('_withOutput','',j)
        if len(list(i.glob(j))) !=0:
            mouse_ids_with_raw_tiff.append(list(i.glob(j)))
#flatten the list and sort
mouse_ids_with_raw_tiff_flat = [item for sublist in mouse_ids_with_raw_tiff for item in sublist]
mouse_ids_with_raw_tiff_flat = sorted(mouse_ids_with_raw_tiff_flat)

#shuffle mouse ids for a 15/4/6 split train/val/test by mouse id
mouse_ids_present = [i.parent.name for i in mouse_ids_with_raw_tiff_flat]
mouse_ids_present = sorted(list(np.unique(mouse_ids_present)))
np.random.seed(643)
np.random.shuffle(mouse_ids_present)
mouse_ids_present
train = mouse_ids_present[:3]
val = mouse_ids_present[-2:]
test = mouse_ids_present[-2:]

train_files = []
val_files = []
test_files = []
for i in train:
    for j in mouse_ids_with_raw_tiff_flat:
        if i in j.as_posix():
            for k in data_dicts:
                if re.sub(".tif","_withOutput.tif",j.name) in k["image"]:
                    train_files.append(k)
for i in val:
    for j in mouse_ids_with_raw_tiff_flat:
        if i in j.as_posix():
            for k in data_dicts:
                if re.sub(".tif","_withOutput.tif",j.name) in k["image"]:
                    val_files.append(k)
for i in test:
    for j in mouse_ids_with_raw_tiff_flat:
        if i in j.as_posix():
            for k in data_dicts:
                if re.sub(".tif","_withOutput.tif",j.name) in k["image"]:
                    test_files.append(k)

[PosixPath('../../../rozakmat/TBI/PV-cap sep model/XYZres042_0001_Simple Segmentation_sub1.tiff'), PosixPath('../../../rozakmat/TBI/PV-cap sep model/XYZres018_0001_Simple Segmentation_sub1.tiff'), PosixPath('../../../rozakmat/TBI/PV-cap sep model/XYZres037_Simple Segmentation_sub1.tiff'), PosixPath('../../../rozakmat/TBI/PV-cap sep model/XYZres050_Simple Segmentation_sub1.tiff'), PosixPath('../../../rozakmat/TBI/PV-cap sep model/XYZres032_Simple Segmentation_sub1.tiff')]
[{'image': '../../../rozakmat/TBI/PV-cap sep model/XYZres018_0001_withOutput.tif', 'label': '../../../rozakmat/TBI/PV-cap sep model/XYZres018_0001_Simple Segmentation_sub1.tiff'}, {'image': '../../../rozakmat/TBI/PV-cap sep model/XYZres032_withOutput.tif', 'label': '../../../rozakmat/TBI/PV-cap sep model/XYZres032_Simple Segmentation_sub1.tiff'}, {'image': '../../../rozakmat/TBI/PV-cap sep model/XYZres037_withOutput.tif', 'label': '../../../rozakmat/TBI/PV-cap sep model/XYZres037_Simple Segmentation_sub1.tiff'}, {'imag

In [19]:
val_files_labels = [x['label'] for x in val_files]

In [20]:
val_files_labels

['../../../rozakmat/TBI/PV-cap sep model/XYZres037_Simple Segmentation_sub1.tiff',
 '../../../rozakmat/TBI/PV-cap sep model/XYZres050_Simple Segmentation_sub1.tiff']

In [22]:
for file in val_files_labels:
    img_gt = io.imread(file)

In [23]:
file

'../../../rozakmat/TBI/PV-cap sep model/XYZres050_Simple Segmentation_sub1.tiff'

In [24]:
img_gt

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       ...,


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]],


       [[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 